In [1]:
pip install cvxpy==1.1.18

     |████████████████████████████████| 2.7 MB 5.0 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [2]:
from sympy import *
import numpy as np


n = 6

x1, x2, x3, x4, x5, x6 = symbols('x1 x2 x3 x4 x5 x6', commutative=False)

g = (x1 + x2 + x3 + x4 + x5 + x6)

beta = ( 1 + g + g**2 ).expand().args 

In [3]:
gamma = [None] * len(beta)

In [4]:
for l in range(len(beta)):
    order = beta[l].args 

    flag = True 
    for _ in range(len( order) ):
        flag = False
        if isinstance(order[_], Integer): 
            flag = True

    if not flag:
        # revert 
        gamma[l] = prod( list ( ( order )[::-1] ) )
    else:
        gamma[l] = beta[l]
            

In [5]:
B1 = [beta[i] * x1 *  gamma[j] for i in range(len(beta)) for j in range(len(beta))]
B2 = [beta[i] * x2 *  gamma[j] for i in range(len(beta)) for j in range(len(beta))] 
B3 = [beta[i] * x3 *  gamma[j] for i in range(len(beta)) for j in range(len(beta))]
B4 = [beta[i] * x4 *  gamma[j] for i in range(len(beta)) for j in range(len(beta))]
B5 = [beta[i] * x5 *  gamma[j] for i in range(len(beta)) for j in range(len(beta))]
B6 = [beta[i] * x6 *  gamma[j] for i in range(len(beta)) for j in range(len(beta))]
B7 = [beta[i] *  gamma[j] for i in range(len(beta)) for j in range(len(beta))]

In [6]:
S = 1 + g +  g**2 + g**3 + g**4 +  g**5

In [7]:
Args = list( S.expand().args )

In [8]:
len(Args)

9331

In [9]:
Args_dict = {}

for i in range(len(Args)):
    Args_dict[Args[i]] = i


def find_basis(B):
    Z =  np.zeros(len(B), dtype=int)
    for l in range(len(B)):
        Z[l] = Args_dict[B[l]]
    return Z
            

In [10]:
Z1 = find_basis(B1).reshape(len(beta), len(beta))
Z2 = find_basis(B2).reshape(len(beta), len(beta))
Z3 = find_basis(B3).reshape(len(beta), len(beta))
Z4 = find_basis(B4).reshape(len(beta), len(beta))
Z5 = find_basis(B5).reshape(len(beta), len(beta))
Z6 = find_basis(B6).reshape(len(beta), len(beta))
Z7 = find_basis(B7).reshape(len(beta), len(beta))

# very slow, should optimize.

In [11]:
U = [Z1, Z2, Z3, Z4, Z5, Z6, Z7]

In [12]:
import itertools

perm = list(itertools.permutations([x1,x2,x3,x4,x5, x6], 4))

In [13]:
T = ( sum([prod(perm[_]) for _ in range(len(perm))]) ).expand().args 

In [14]:
G = find_basis(T)

In [15]:
A = [None] * len(Args)

for index in range(len(Args)):
    A[index] = np.zeros( ( (n+1) * len(beta) + 1,  (n+1) * len(beta) + 1) )
    M = np.zeros((len(beta), len(beta)))
    for k in range(n+1):
        start = 1 + k * len(beta)
        final = 1 + (k + 1) * len(beta)
        P = U[k] == index
        if k < n:
            M += P
            A[index][ start:final, start:final ] = P
        else:
            A[index][ start:final, start:final ] = n * P - M

In [16]:
A[0][0, 0] = -1

In [17]:
import cvxpy as cp
import numpy as np

# Generate a random SDP.
p = len(Args)

C = np.zeros(((n+1) * len(beta) + 1, (n+1) * len(beta) + 1))
C[0,0]=1
b = np.zeros(p)
b[G] = -1

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable(((n+1) * len(beta) + 1, (n+1) * len(beta) + 1), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve(verbose=True)

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

                                     CVXPY                                     
                                    v1.1.18                                    
(CVXPY) Jan 23 05:36:33 AM: Your problem has 91204 variables, 9332 constraints, and 0 parameters.
(CVXPY) Jan 23 05:36:34 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 23 05:36:34 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 23 05:36:34 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 23 05:36:35 AM: Compiling problem (target solver=SCS).
(CVXPY) Jan 23 05:36:35 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

In [22]:
import cvxpy as cp
import numpy as np

# Generate a random SDP.
p = len(Args)

C = np.zeros(((n+1) * len(beta) + 1, (n+1) * len(beta) + 1))
C[0,0]=1
b = np.zeros(p)
b[G] = 1

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable(((n+1) * len(beta) + 1, (n+1) * len(beta) + 1), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve(verbose=True)

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

                                     CVXPY                                     
                                    v1.1.18                                    
(CVXPY) Jan 23 05:53:27 AM: Your problem has 91204 variables, 9332 constraints, and 0 parameters.
(CVXPY) Jan 23 05:53:27 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 23 05:53:27 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 23 05:53:27 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 23 05:53:28 AM: Compiling problem (target solver=SCS).
(CVXPY) Jan 23 05:53:28 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi